In [4]:
import tensorflow as tf
import numpy as np
import pickle
#import matplotlib.pyplot as plt
import timeit
import time
from glob import glob
from scipy import misc
import os, cv2, random
#import pandas as pd
from AlexNetHelper import AlexNetHelper

<h1>Load training, validation, testing set from your preprocessed files</h1>

In [5]:
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR) #cv2.IMREAD_GRAYSCALE
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)


def prep_data(images):
    count = len(images)
    data = np.ndarray((count, CHANNELS, ROWS, COLS), dtype=np.uint8)

    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image.T
        if i%250 == 0: print('Processed {} of {}'.format(i, count))

    return data

In [9]:
TRAIN_DIR = 'C:/Users/Xiangyu Gao/Downloads/train/'
TEST_DIR = 'C:/Users/Xiangyu Gao/Downloads/test1/'

ROWS = 227
COLS = 227
CHANNELS = 3
print(os.listdir(TRAIN_DIR)[:5])
train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] # use this for full dataset
train_dogs =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'dog' in i]
train_cats =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'cat' in i]

test_images =  [TEST_DIR+i for i in os.listdir(TEST_DIR)]


# slice datasets for memory efficiency on Kaggle Kernels, delete if using full dataset
train_images = train_dogs[:] + train_cats[:]
random.shuffle(train_images)
test_images =  test_images[:]

train_labels = []

for i in train_images:
    if 'dog' in i.split('/')[-1]:
        train_labels.append([1,0])
    else:
        train_labels.append([0,1])

#sns.countplot(labels).set_title('Cats and Dogs')
train_labels = np.array(train_labels)
# 

train_images = prep_data(train_images)
test_images = prep_data(test_images)
train_images = np.swapaxes(train_images, 1,3)
test_images = np.swapaxes(test_images, 1,3)

print("Train shape: {}".format(train_images.shape))
print("Test shape: {}".format(test_images.shape))


['cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.1000.jpg']
Processed 0 of 25000
Processed 250 of 25000
Processed 500 of 25000
Processed 750 of 25000
Processed 1000 of 25000
Processed 1250 of 25000
Processed 1500 of 25000
Processed 1750 of 25000
Processed 2000 of 25000
Processed 2250 of 25000
Processed 2500 of 25000
Processed 2750 of 25000
Processed 3000 of 25000
Processed 3250 of 25000
Processed 3500 of 25000
Processed 3750 of 25000
Processed 4000 of 25000
Processed 4250 of 25000
Processed 4500 of 25000
Processed 4750 of 25000
Processed 5000 of 25000
Processed 5250 of 25000
Processed 5500 of 25000
Processed 5750 of 25000
Processed 6000 of 25000
Processed 6250 of 25000
Processed 6500 of 25000
Processed 6750 of 25000
Processed 7000 of 25000
Processed 7250 of 25000
Processed 7500 of 25000
Processed 7750 of 25000
Processed 8000 of 25000
Processed 8250 of 25000
Processed 8500 of 25000
Processed 8750 of 25000
Processed 9000 of 25000
Processed 9250 of 25000
Processed 9500 of 2500

In [10]:
vali_images = train_images[20000:25000]
vali_labels = train_labels[20000:25000]
train_images = train_images[0:20000]
train_labels = train_labels[0:20000]

In [11]:
print(np.shape(train_labels))
print(np.shape(vali_labels))
print(np.shape(train_images))
print(np.shape(vali_images))

(20000, 2)
(5000, 2)
(20000, 227, 227, 3)
(5000, 227, 227, 3)


In [12]:
#train_images, train_labels, valid_images = get_data()
num_classes = train_labels.shape[1]
print(np.shape(train_labels))

(20000, 2)


In [19]:
train_images = train_images[0:10000]
train_labels = train_labels[0:10000]

<h1>Define hyperparameter</h1>

In [23]:
batch_size = 16
epochs = 10
lr = 0.01

In [14]:
print(np.shape(train_images))

(20000, 227, 227, 3)


<h1>Placeholder</h1>

In [15]:
tf.reset_default_graph()
# images = tf.placeholder(tf.float32, shape=[None, *train[0].shape])
# labels = tf.placeholder(tf.float32, shape=[None, 2])

<h1>AlexNet</h1>

In [17]:
def inference(images, num_classes):
        """Builds AlexNet Graph

        Args:
            images: train/test images
            num_classes: number of classes
        Returns:
            final_node: last node in the graph
        """

        # Layer 1
        with tf.name_scope("conv1") as scope:
            kernel = AlexNetHelper.instintiate_weights('W1', [11, 11, 3, 96])
            conv = tf.nn.conv2d(images, kernel, [1, 4, 4, 1], padding='SAME')
            biases = AlexNetHelper.instintiate_bias('b1', [96])
            conv1 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        with tf.name_scope('batch_norm1') as scope:
            mean, var = tf.nn.moments(conv1, [0, 1, 2])
            batch_norm1 = tf.nn.batch_normalization(conv1, mean, var, 0, 1, 0, name=scope)

        with tf.name_scope('pool1') as scope:
            pool1 = tf.nn.max_pool(batch_norm1, [1, 3, 3, 1], [1, 2, 2, 1], padding='SAME', name=scope)

        # Layer 2
        with tf.name_scope("conv2") as scope:
            kernel = AlexNetHelper.instintiate_weights('W2', [5, 5, 96, 256])
            conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = AlexNetHelper.instintiate_bias('b2', [256])
            conv2 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        with tf.name_scope('batch_norm2') as scope:
            mean, var = tf.nn.moments(conv2, [0, 1, 2])
            batch_norm2 = tf.nn.batch_normalization(conv2, mean, var, 0, 1, 0, name=scope)

        with tf.name_scope('pool2') as scope:
            pool2 = tf.nn.max_pool(batch_norm2, [1, 3, 3, 1], [1, 2, 2, 1], padding='SAME', name=scope)

        # Layer 3
        with tf.name_scope("conv3") as scope:
            kernel = AlexNetHelper.instintiate_weights('W3', [3, 3, 256, 384])
            conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = AlexNetHelper.instintiate_bias('b3', [384])
            conv3 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        with tf.name_scope('batch_norm3') as scope:
            mean, var = tf.nn.moments(conv3, [0, 1, 2])
            batch_norm3 = tf.nn.batch_normalization(conv3, mean, var, 0, 1, 0, name=scope)

        # Layer 4
        with tf.name_scope("conv4") as scope:
            kernel = AlexNetHelper.instintiate_weights('W4', [3, 3, 384, 384])
            conv = tf.nn.conv2d(batch_norm3, kernel, [1, 1, 1, 1], padding='SAME')
            biases = AlexNetHelper.instintiate_bias('b4', [384])
            conv4 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        with tf.name_scope('batch_norm4') as scope:
            mean, var = tf.nn.moments(conv4, [0, 1, 2])
            batch_norm4 = tf.nn.batch_normalization(conv4, mean, var, 0, 1, 0, name=scope)

        # Layer 5
        with tf.name_scope("conv5") as scope:
            kernel = AlexNetHelper.instintiate_weights('W5', [3, 3, 384, 256])
            conv = tf.nn.conv2d(batch_norm4, kernel, [1, 1, 1, 1], padding='SAME')
            biases = AlexNetHelper.instintiate_bias('b5', [256])
            conv5 = tf.nn.relu(tf.nn.bias_add(conv, biases), name=scope)

        with tf.name_scope('batch_norm5') as scope:
            mean, var = tf.nn.moments(conv5, [0, 1, 2])
            batch_norm5 = tf.nn.batch_normalization(conv5, mean, var, 0, 1, 0, name=scope)

        with tf.name_scope('pool5') as scope:
            pool5 = tf.nn.max_pool(batch_norm5, [1, 3, 3, 1], [1, 2, 2, 1], padding='SAME', name=scope)

        # Fully Connected 6
        with tf.name_scope('FC6') as scope:
            pool5_flat = tf.contrib.layers.flatten(pool5)
            fc6 = tf.layers.dense(pool5_flat, units=4096, activation=tf.nn.relu, name=scope)
            # fc6_dropout = tf.layers.dropout(fc6, 0.5)

        with tf.name_scope('FC7') as scope:
            fc7 = tf.layers.dense(fc6, units=4096, activation=tf.nn.relu, name=scope)
            # fc7_dropout = tf.layers.dropout(fc7, 0.5)

        with tf.name_scope('classes') as scope:
            predictions = tf.layers.dense(fc7, units=num_classes, activation=tf.nn.softmax, name=scope)

        return predictions

<h1>Cost and Optimization</h1>

<h1>Training and validation</h1>
<h2>Train your model only 10 epochs</h2>
<p style="font-size:20px">1. Print out training accuracy and validation accuracy each training epoch</p>
<p style="font-size:20px">2. Print out training time each training epoch</p>
<p style="font-size:20px">3. Your goal is to reach 85% validation accuracy in 10 training epochs. If you reach that, you can perform testing, print out your test accuracy. Plot out the ten images with title that contains the probability of the labeled class.</p>

In [24]:
"""
Runtime for AlexNet
:param
train_data: training data
num_classes: num_classes
:return: None
"""
print(np.shape(train_images))
print(np.shape(train_labels))
tf.reset_default_graph()
num_classes = train_labels.shape[1]

images = tf.placeholder(tf.float32, shape=[None, 227,227,3])
labels = tf.placeholder(tf.float32, shape=[None, 2])

predictions = inference(images, num_classes)
init = tf.initialize_all_variables()

loss = tf.reduce_mean(tf.nn.l2_loss(tf.square(predictions - labels)))
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = predictions, labels = ph_labels))
learner = tf.train.GradientDescentOptimizer(learning_rate = 0.001)
optimizer = learner.minimize(loss)

correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for ep in range(epochs):
        l_total = 0
        acc = 0
        start_time = time.time()
        for i in range(0, len(train_images), batch_size):
            sess.run(optimizer, feed_dict={images: train_images[i: batch_size + i,:], \
                                            labels: train_labels[i: batch_size + i,:]})
            #print(i)
        use_time = time.time() - start_time
        acc = sess.run(accuracy, feed_dict={images:vali_images, labels:vali_labels})
        print("EPOCH {0}, training time {1:.4f}s, Validation Accuracy = {2:.3f}".format(ep, use_time, acc))
        # val_acc, _ = sess.run([accuracy, predictions],
        #                        feed_dict={images: valid_images, labels: valid_labels})
        # print("Validation accuracy: ", val_acc)

(10000, 227, 227, 3)
(10000, 2)


ResourceExhaustedError: OOM when allocating tensor with shape[5000,3,227,227] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: conv1/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_Placeholder_0_0/_71, PermConstNHWCToNCHW-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: Mean_1/_75 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_208_Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation